In [ ]:
#$ export FLASK_APP=endpoints.py
#$ flask run

In [1]:
import requests
from sql_caller import engine_connector, user_call
from sqlalchemy import text
import pandas as pd

In [ ]:
@app.route("/")
def hello_world():
    return {'hello':'world'}

@app.route('/name')
def te_doy_tu_nombre():
    name = request.args.get('name')
    lastname = request.args.get('lastname')
    return {"Hello": f"{name} {lastname}"}

@app.route('/mucho')
def mucho():
    args = request.args
    res = {}
    for k, v in args.items():
        res[k] = v
    return res

